<a href="https://colab.research.google.com/github/fernandomarca/IA/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do java

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Instalação do spark

In [20]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!tar -xvf spark-3.3.0-bin-hadoop3.tgz

In [22]:
!pip install -q findspark

In [23]:
!pip install -q pyspark

     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 40.0 MB/s 


In [26]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.3.0-bin-hadoop3'

In [27]:
import findspark
findspark.init()

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [29]:
dataset = spark.read.csv('/content/sample_data/california_housing_test.csv',inferSchema=True, header =True)

In [30]:
dataset.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [33]:
from pyspark import SparkContext 
print(spark.sparkContext)           
print(spark.version)

<SparkContext master=local[*] appName=pyspark-shell>
3.3.0


In [34]:
dataset.head()

Row(longitude=-122.05, latitude=37.37, housing_median_age=27.0, total_rooms=3885.0, total_bedrooms=661.0, population=1537.0, households=606.0, median_income=6.6085, median_house_value=344700.0)

In [35]:
dataset.count()

3000

In [36]:
dataset.createOrReplaceTempView('tabela_temporaria')
print(spark.catalog.listTables())

[Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [37]:
query = 'FROM tabela_temporaria SELECT longitude, latitude LIMIT 3'  
saida = spark.sql(query)  
saida.show() 

+---------+--------+
|longitude|latitude|
+---------+--------+
|  -122.05|   37.37|
|   -118.3|   34.26|
|  -117.81|   33.78|
+---------+--------+



In [38]:
query1 = 'SELECT MAX(total_rooms) as maximo_quartos FROM tabela_temporaria'
q_maximo_quartos = spark.sql(query1)
pd_maximo_quartos = q_maximo_quartos.toPandas()
print('A quantidade máxima de quartos é: {}'.format(pd_maximo_quartos['maximo_quartos']))
qtd_maximo_quartos = int(pd_maximo_quartos.loc[0,'maximo_quartos'])

A quantidade máxima de quartos é: 0    30450.0
Name: maximo_quartos, dtype: float64


In [43]:
query2 = 'SELECT longitude, latitude FROM tabela_temporaria WHERE total_rooms = '+str(qtd_maximo_quartos)
localizacao_maximo_quartos = spark.sql(query2)
pd_localizacao_maximo_quartos = localizacao_maximo_quartos.toPandas()
print(pd_localizacao_maximo_quartos)

   longitude  latitude
0     -117.2     33.58


In [44]:
import pandas as pd
import numpy as np
media = 0
desvio_padrao=0.1 
pd_temporario = pd.DataFrame(np.random.normal(media,desvio_padrao,100))
spark_temporario = spark.createDataFrame(pd_temporario)
print(spark.catalog.listTables())
spark_temporario.createOrReplaceTempView('nova_tabela_temporaria')
print(spark.catalog.listTables())

[Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [45]:
spark_temporario.show()

+--------------------+
|                   0|
+--------------------+
|  0.0433451970739987|
|-0.04387326485753...|
|-0.17679736587438197|
| 0.22591325121363198|
| 0.14889810218007063|
|0.014318939198778186|
| 0.18823651256663626|
|0.006224796408807163|
| 0.18212583684609224|
|  0.0638879933478317|
| -0.0264840172196403|
|-0.00475935448094...|
|0.039678404620187596|
|-0.13695215642062644|
|0.015343283591580034|
| 0.04032363415304467|
| 0.07109187215371231|
| 0.18861156036300591|
| 0.07706713251228277|
|   0.139707745031223|
+--------------------+
only showing top 20 rows



In [60]:
vetor = np.array([10, 20, 30, 40, 50])
print(vetor)

[10 20 30 40 50]


In [61]:
spark_contexto = spark.sparkContext

In [62]:
paralelo = spark_contexto.parallelize(vetor)

In [52]:
print(paralelo)

ParallelCollectionRDD[69] at readRDDFromFile at PythonRDD.scala:274


In [63]:
mapa = paralelo.map(lambda x : x**2+x)

In [64]:
print(mapa.collect())

[110, 420, 930, 1640, 2550]


In [65]:
from operator import add
soma = mapa.reduce(add)

In [66]:
print(soma)

5650


In [56]:
paralelo = spark_contexto.parallelize(['distribuida', 'distribuida', 'spark', 'rdd', 'spark','spark'])

In [57]:
funcao_lambda = lambda x:(x,1)

In [58]:
from operator import add
mapa = paralelo.map(funcao_lambda).reduceByKey(add).collect()

In [59]:
for (w, c) in mapa:
  print('{}: {}'.format(w, c))

distribuida: 2
spark: 3
rdd: 1
